## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Banda Aceh,ID,5.5577,95.3222,79.59,80,100,5.64,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,55.40,67,0,11.50,clear sky
2,2,Victoria,HK,22.2855,114.1577,73.00,83,99,1.01,overcast clouds
3,3,Nikolskoye,RU,59.7035,30.7861,42.01,70,0,6.71,clear sky
4,4,Bayonet Point,US,28.3267,-82.6834,80.01,47,16,5.01,few clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164 entries, 1 to 684
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              164 non-null    int64  
 1   City                 164 non-null    object 
 2   Country              160 non-null    object 
 3   Lat                  164 non-null    float64
 4   Lng                  164 non-null    float64
 5   Max Temp             164 non-null    float64
 6   Humidity             164 non-null    int64  
 7   Cloudiness           164 non-null    int64  
 8   Wind Speed           164 non-null    float64
 9   Current Description  164 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 14.1+ KB


In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Punta Arenas,CL,55.40,clear sky,-53.1500,-70.9167,
5,Hermanus,ZA,64.99,broken clouds,-34.4187,19.2345,
14,Airai,TL,59.40,scattered clouds,-8.9266,125.4092,
16,Sayyan,YE,60.62,clear sky,15.1718,44.3244,
20,New Norfolk,AU,60.80,light rain,-42.7826,147.0587,
21,Zhezkazgan,KZ,50.97,overcast clouds,47.8043,67.7144,
23,Hobart,AU,60.80,broken clouds,-42.8794,147.3294,
34,Port Alfred,ZA,66.31,overcast clouds,-33.5906,26.8910,
36,Waterloo,CA,66.00,clear sky,43.4668,-80.5164,
38,Busselton,AU,60.01,overcast clouds,-33.6500,115.3333,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.... Skipping.")
        
print("Done")      

Hotel not found.... Skipping.
Hotel not found.... Skipping.
Hotel not found.... Skipping.
Hotel not found.... Skipping.
Hotel not found.... Skipping.
Hotel not found.... Skipping.
Done


In [23]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Punta Arenas,CL,55.40,clear sky,-53.1500,-70.9167,Hotel Dreams Del Estrecho
5,Hermanus,ZA,64.99,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
14,Airai,TL,59.40,scattered clouds,-8.9266,125.4092,Juvinal Martin
16,Sayyan,YE,60.62,clear sky,15.1718,44.3244,متنزه و بحيرة سيان
20,New Norfolk,AU,60.80,light rain,-42.7826,147.0587,The Shingles Riverside Cottages


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))